## Imports

Most recent version of spotipy

Uncomment this if you haven't installed it yet

In [29]:
# !pip install git+https://github.com/plamere/spotipy.git --upgrade

In [30]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import json
import os

## Spotipy setup

- Need client id and client secret which you can get from [here](https://developer.spotify.com/dashboard/login).
- Also need to setup your redirectURI. I just use Google. It doesn't really matter. You can do this by clicking on the green 'Edit Settings' button. Make sure to add and save.
- Username is your user id which you can find by (at least on the desktop app) clicking on your name in the top right to get to your profile, clicking the '...' under your name, hover over 'Share' and navigate to 'Copy Profile Link'. Click it. Paste somewhere to show the link which should be something like https://open.spotify.com/user/** where ** is your user id.

In [31]:
cId = "xxx"
cSecret = "xxx"
redirectURI = "https://google.com/"
username = "xxx"

client_credentials_manager = SpotifyClientCredentials(client_id=cId, client_secret=cSecret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = ''
token = util.prompt_for_user_token(username, scope, cId, cSecret, redirectURI)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

## Importing some saved tracks from spotify to a csv file

Get US top 50 playlist id. It should be 37i9dQZEVXbLRQDuF5jeBp.

In [32]:
us_top50_id = ''
limit = 50
for i in range(1100, 1300, limit):
    playlists = sp.user_playlists('spotify', limit=limit, offset=i)
    for pl in playlists['items']:
        if pl['name'] == 'United States Top 50':
            us_top50_id = pl['id']
            break
#     if not us_top50_id:
#         break
us_top50_id

'37i9dQZEVXbLRQDuF5jeBp'

This stores the user's saved tracks, artists, and albums into their respective lists.

In [1]:
# print(json.dumps(top_us_50_songs,indent=True))

In [35]:
track_list = []
artist_list = []
album_list = []

song_artist_list = []
song_album_list = []
album_artist_list = []

top_us_50_songs = sp.user_playlist('spotify', playlist_id=us_top50_id, fields=None)

for t in top_us_50_songs['tracks']['items']:
    track = t['track']
    track_id = track['id']
    track_name = track['name']
    track_link = track['external_urls']['spotify']

    # iterating through each artist of the track
    for artist in track['artists']:
        artist_id = artist['id']

        # storing artist in artist list
        artist_dict = {
            'artist_id': artist_id,
            'artist_name': artist['name']
        }
        artist_list.append(artist_dict)

        # storing track id and artist id in list
        song_artist_dict = {
            'track_id': track_id,
            'artist_id': artist_id
        }
        song_artist_list.append(song_artist_dict)

    # getting album info
    track_album = track['album']
    album_id = track_album['id']
    album_name = track_album['name']
    album_img = track_album['images'][0]['url']
    album_type = track_album['album_type']
    release_date = track_album['release_date']
    total_tracks = track_album['total_tracks']

    # iterating through album artists
    for album in track_album['artists']:
        # storing album id and artist id in list
        album_artist_dict = {
            'album_id': album_id,
            'artist_id': album['id']
        }
        album_artist_list.append(album_artist_dict)

    # storing track id and album id in list
    song_album_dict = {
        'track_id': track_id,
        'album_id': album_id
    }
    song_album_list.append(song_album_dict)

    # storing track info in list
    track_dict = {
        'track_id': track_id,
        'track_name': track_name,
        'track_link': track_link
    }
    track_list.append(track_dict)

    # storing album info in list
    album_dict = {
        'album_id': album_id,
        'album_name': album_name,
        'album_img': album_img,
        'album_type': album_type,
        'release_date': release_date,
        'total_tracks': total_tracks
    }
    album_list.append(album_dict)

In [37]:
# Converting the track list into a dataframe

track_df = pd.DataFrame(track_list)
track_df.head() # preview first 5 rows of data

,track_id,track_link,track_name
0,2JvzF1RMd7lE3KmFlsyZD8,https://open.spotify.com/track/2JvzF1RMd7lE3Km...,MIDDLE CHILD
1,3KkXRkHbMCARz0aVfEt68P,https://open.spotify.com/track/3KkXRkHbMCARz0a...,Sunflower - Spider-Man: Into the Spider-Verse
2,6ocbgoVGwYJhOv1GgI9NsF,https://open.spotify.com/track/6ocbgoVGwYJhOv1...,7 rings
3,4y3OI86AEP6PQoDE6olYhO,https://open.spotify.com/track/4y3OI86AEP6PQoD...,Sucker
4,7eBqSVxrzQZtK2mmgRG6lC,https://open.spotify.com/track/7eBqSVxrzQZtK2m...,Murder On My Mind


In [23]:
artist_df = pd.DataFrame(artist_list)
artist_df.drop_duplicates(keep='first', inplace=True)
# artist_df.head()

In [24]:
album_df = pd.DataFrame(album_list)
album_df.drop_duplicates(keep='first', inplace=True) # drop duplicate albums
# album_df.head()

In [25]:
song_artist_df = pd.DataFrame(song_artist_list)
# song_artist_df.set_index('track_id', inplace=True)

song_album_df = pd.DataFrame(song_album_list)
# song_album_df.set_index('track_id', inplace=True)

album_artist_df = pd.DataFrame(album_artist_list)
# album_artist_df.set_index('album_id', inplace=True)
album_artist_df.drop_duplicates(keep='first', inplace=True)

Checking dataframe shapes

In [26]:
print('track', track_df.shape)
print('artist', artist_df.shape)
print('album', album_df.shape)
print('song_artist', song_artist_df.shape)
print('song_album', song_album_df.shape)
print('album_artist', album_artist_df.shape)

track (50, 3)
artist (51, 2)
album (43, 6)
song_artist (73, 2)
song_album (50, 2)
album_artist (54, 2)


Saving the dataframes as csv files

In [27]:
csv_folder_name = 'data/csv/'
if not os.path.exists(csv_folder_name):
    os.makedirs(csv_folder_name)

track_df.to_csv(csv_folder_name + 'songs.csv')
artist_df.to_csv(csv_folder_name + 'artists.csv')
album_df.to_csv(csv_folder_name + 'albums.csv')

song_artist_df.to_csv(csv_folder_name + 'song-artist.csv')
song_album_df.to_csv(csv_folder_name + 'song-album.csv')
album_artist_df.to_csv(csv_folder_name + 'album-artist.csv')

Create .dat files with
- no headers (a.k.a. column names)
- columns deliminated by vertical bars

Created for SQL db.

Note: .dat extension was made up

In [28]:
dat_folder_name = 'data/dat/'
if not os.path.exists(dat_folder_name):
    os.makedirs(dat_folder_name)

track_df.to_csv(dat_folder_name + 'songs.dat', sep='|', header=False, index=False)
artist_df.to_csv(dat_folder_name + 'artists.dat', sep='|', header=False, index=False)
album_df.to_csv(dat_folder_name + 'albums.dat', sep='|', header=False, index=False)

song_artist_df.to_csv(dat_folder_name + 'song-artist.dat', sep='|', header=False, index=False)
song_album_df.to_csv(dat_folder_name + 'song-album.dat', sep='|', header=False, index=False)
album_artist_df.to_csv(dat_folder_name + 'album-artist.dat', sep='|', header=False, index=False)

Check out both folders!

Run SQL script with ***source initDB.sql*** in netlab after transferring files.